In [1]:
import os
import sys  
from pathlib import Path
from typing import Union
import torch
import cv2 as cv
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort

sys.path.append(os.path.abspath('C:/Users/Shahab/Downloads/Jupyter/Car-license-Plate/My-Code/Project/yolov7'))
from models.experimental import attempt_load
from utils.general import check_img_size
from utils.torch_utils import select_device, TracedModel
from utils.datasets import letterbox
from utils.general import non_max_suppression, scale_coords
from utils.plots import plot_one_box, plot_one_box_PIL

from copy import deepcopy
import easyocr
import matplotlib.pyplot as plt
from math import sqrt, atan, degrees
from tensorflow.keras.models import load_model  

In [2]:
weights = 'C:/Users/Shahab/Downloads/Jupyter/Car-license-Plate/My-Code/Project/yolov7/weights/best.pt'
device_id = 'cpu'   #'cuda'
image_size = 640
trace = True

# بارگذاری مدل  
model_number = load_model('C:/Users/Shahab/Downloads/Jupyter/Car-license-Plate/My-Code/Project/model/model_number.h5') 
model_alphabet = load_model('C:/Users/Shahab/Downloads/Jupyter/Car-license-Plate/My-Code/Project/model/model_alphabet.h5')  

# Initialize
device = select_device(device_id)
half = device.type != 'cpu'  # half precision only supported on CUDA

# Load model
model = attempt_load(weights, map_location=device)  # load FP32 model
stride = int(model.stride.max())  # model stride
imgsz = check_img_size(image_size, s=stride)  # check img_size

#The model is tracked for optimization.
if trace:
    model = TracedModel(model, device, image_size)

# Only supported on CUDA
# Faster calculations
if half:
    model.half()  # to FP16

# If a GPU is used, the model is run once to perform an initial warm-up operation.
if device.type != 'cpu':
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once

# Load OCR
#reader = easyocr.Reader(['fa'])

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


C:\Users\Shahab\qenv\lib\site-packages\torch\functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:3638.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



In [3]:
def detect_plate(source_image):
    # Padded resize
    img_size = 640
    stride = 32
    img = letterbox(source_image, img_size, stride=stride)[0]

    # Convert
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)
        
    with torch.no_grad():
        # Inference
        pred = model(img, augment=True)[0]

    # Apply NMS
    pred = non_max_suppression(pred, 0.25, 0.45, classes=0, agnostic=True)

    plate_detections = []
    det_confidences = []
    
    # Process detections
    for i, det in enumerate(pred):  # detections per image
        if len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], source_image.shape).round()

            # Return results
            for *xyxy, conf, cls in reversed(det):
                coords = [int(position) for position in (torch.tensor(xyxy).view(1, 4)).tolist()[0]]
                plate_detections.append(coords)
                det_confidences.append(conf.item())

    return plate_detections, det_confidences

"""
def unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=2.0, threshold=0):
    blurred = cv.GaussianBlur(image, kernel_size, sigma)
    sharpened = float(amount + 1) * image - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharpened, image, where=low_contrast_mask)
    return sharpened
"""

def crop(image, coord):
    cropped_image = image[int(coord[1]):int(coord[3]), int(coord[0]):int(coord[2])]
    return cropped_image

"""
counter_ocr_plate = 0
def ocr_plate(plate_region):
    global counter_ocr_plate
    # Image pre-processing for more accurate OCR
    #cv.imwrite(os.path.join(savepath, "plate_img.png"), plate_region)
    rescaled = cv.resize(plate_region, None, fx=1.2, fy=1.2, interpolation=cv.INTER_CUBIC)
    grayscale = cv.cvtColor(rescaled, cv.COLOR_BGR2GRAY)
    # OCR the preprocessed image
    grayscale_blur = cv.medianBlur(grayscale, 1)
    ret, thresh1 = cv.threshold(grayscale_blur, 120, 255, cv.THRESH_BINARY + cv.THRESH_OTSU) 
    cv.imwrite(os.path.join(savepath, "grayscale_blur%s.png" %counter_ocr_plate), grayscale_blur)
    counter_ocr_plate += 1
    plate_text_easyocr = reader.readtext(grayscale_blur)
    if plate_text_easyocr:
        (bbox, text_easyocr, ocr_confidence) = plate_text_easyocr[0]
        #print("plate_text Easyocr ", text_easyocr)
    else:
        text_easyocr = "_"
        ocr_confidence = 0
    #if ocr_confidence == 'nan':
    
    return text_easyocr, ocr_confidence
"""

#Rotate Pelak
def find_longest_line(plate_img_gr):
    kernel_size = 3
    blur_gray = cv.GaussianBlur(plate_img_gr, (kernel_size, kernel_size), 0)

    low_threshold = 150
    high_threshold = 200

    edges = cv.Canny(blur_gray, low_threshold, high_threshold)

    rho = 1  # distance resolution in pixels of the Hough grid
    theta = np.pi / 180  # angular resolution in radians of the Hough grid
    threshold = 15  # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 50  # minimum number of pixels making up a line
    max_line_gap = 5  # maximum gap in pixels between connectable line segments
    line_image = np.copy(plate_img_gr) * 0  # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)

    
    lls = []
    for indx, line in enumerate(lines):
        for x1,y1,x2,y2 in line:
            cv.line(line_image,(x1,y1),(x2,y2),(255,0,0),5)
            line_length = sqrt((x2-x1)**2 + (y2-y1)**2)
            lls.append((indx,line_length))
    lls.sort(key = lambda x: x[1])
    linessorted = []
    for (indx,ll) in lls:
        linessorted.append(lines[indx])
    return linessorted

def find_line_angle(line):
    x1,y1,x2,y2 = line[0]
    angle = degrees(atan(((y2-y1)/(x2-x1))))
    return angle

def rotate_image(plate_img_gr, angle):
    (h, w) = plate_img_gr.shape
    (cX, cY) = (w // 2, h // 2)
    M = cv.getRotationMatrix2D((cX, cY), angle, 1.0)
    rotated = cv.warpAffine(plate_img_gr, M, (w, h))
    return rotated

def adjust_cropping(rotated_img):
    h,w = rotated_img.shape
    targ_h = int(w/4)
    crop_h = int((h - targ_h)/2)
    cropped_rotated_img = rotated_img[crop_h:h-crop_h,:]
    return cropped_rotated_img

#تیکه تیکه کردن پلاک


def Cutting_Pieces (plate_region,pcc):
    h, w = plate_region.shape
    chopfactors = [(60,125), (125,190), (180,300), (280,350), (340,415), (400,465), (460,535), (515,600)]
    plate_letters = []
    c = 1
    for factor in chopfactors:
        w1 = int((factor[0]/600)*w)
        w2 = int((factor[1]/600)*w)
        letterpatch = plate_region[:,w1:w2]
        letterpatch_resized  = cv.resize(letterpatch, (28,28), interpolation = cv.INTER_LINEAR)
        #ax[c].imshow(letterpatch_resized, cmap='gray')
        cv.imwrite(os.path.join(savepath, "pieces/%s-%s.png" %(pcc,c)), letterpatch_resized)
        c += 1
        plate_letters.append(letterpatch_resized)
    return plate_letters

cont = 5100
#Model Number and alphabet
def pred_number_alphabet(list_img):
    global model_alphabet
    global model_number
    global cont
    lst = []
    c = 0
    for i in list_img:
        img = i.copy()
        if c != 2: 
            i = i / 255
            i = np.array([i])
            y_pred = model_number.predict(i)
            result = y_pred.argmax()
            lst.append(str(result))
        else:
            i = i / 255
            i = np.array([i])
            y_pred = model_alphabet.predict(i)
            result = y_pred.argmax()
            if result == 10:
                lst.append('B')
            elif result == 11:
                lst.append('Q')
            elif result == 12:
                lst.append('H')
            elif result == 13:
                lst.append('SAD')
            elif result == 14:
                lst.append('TA')
            elif result == 15:
                lst.append('V')
            elif result == 16:
                lst.append('M')
            elif result == 17:
                lst.append('J')
            elif result == 18:
                lst.append('Y')
            elif result == 19:
                lst.append('EIN')
            elif result == 20:
                lst.append('N')
            elif result == 21:
                lst.append('D')
            elif result == 22:
                lst.append('T')
            elif result == 23:
                lst.append('HE')
            elif result == 24:
                lst.append('L')
            elif result == 25:
                lst.append('SIN')
        cv.imwrite(os.path.join(savepath, "pred/%s-%s.png" %(cont,result)), img)
        cont += 1
            
        c += 1
    print(lst)
    return lst


ro = 1
pc = 1
# Main function
def get_plates_from_image(input):
    if input is None:
        return None
    global ro
    global pc
    plate_detections, det_confidences = detect_plate(input)
    detected_image = deepcopy(input)
    
    for coords in plate_detections:
        try:
            plate_region = crop(input, coords)
            #Rotate Pelak
            plate_img = plate_region.copy()
            plate_img_gr = cv.cvtColor(plate_region, cv.COLOR_BGR2GRAY)
            
            try:
                linessorted = find_longest_line(plate_img_gr)
                rot_angle = find_line_angle(linessorted[-1])
                rotated_img = rotate_image(plate_img_gr, rot_angle)
                cropped_rotated_img = adjust_cropping(rotated_img)
                w, h = cropped_rotated_img.shape
                if w < 10 or h < 10:
                    cv.imwrite(os.path.join(savepath, "license-plate/%s.png" %ro), plate_img_gr)
                    ro += 1
                    list_plate_letter = Cutting_Pieces(plate_img_gr, pc)
                else :
                    cv.imwrite(os.path.join(savepath, "license-plate/%s.png" %ro), cropped_rotated_img)
                    ro += 1
                    #تیکه تیکه کردن عکس 
                    list_plate_letter = Cutting_Pieces(cropped_rotated_img, pc)
                
            except:
                #تیکه تیکه کردن عکس
                cv.imwrite(os.path.join(savepath, "license-plate/%s.png" %ro), plate_img_gr)
                ro += 1
                list_plate_letter = Cutting_Pieces(plate_img_gr, pc)
            pc += 1
            #پیشبینی اعداد
            y_preds = pred_number_alphabet(list_plate_letter)
            str_y_preds = ''.join(y_preds)
            detected_image = plot_one_box_PIL(coords, detected_image, label=str_y_preds, color=[0, 150, 255], line_thickness=2)
        
        except:
            print('Error')
            continue
            
    return detected_image

"""
def pascal_voc_to_coco(x1y1x2y2):
    x1, y1, x2, y2 = x1y1x2y2
    return [x1, y1, x2 - x1, y2 - y1]
"""

"""
def get_best_ocr(preds, rec_conf, ocr_res, track_id):
    for info in preds:
    # Check if it is current track id
        if info['track_id'] == track_id:
          # Check if the ocr confidenence is maximum or not
            if info['ocr_conf'] < rec_conf:
                info['ocr_conf'] = rec_conf
                info['ocr_txt'] = ocr_res
            else:
                rec_conf = info['ocr_conf']
                ocr_res = info['ocr_txt']
            break
    return preds, rec_conf, ocr_res
"""

"""
def get_plates_from_video(source):
    if source is None:
        return None
    
    # Create a VideoCapture object
    video = cv.VideoCapture(source)

    # Default resolutions of the frame are obtained. The default resolutions are system dependent.
    # We convert the resolutions from float to integer.
    width = int(video.get(cv.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv.CAP_PROP_FPS)

    # Define the codec and create VideoWriter object.
    temp = f'{Path(source).stem}_temp{Path(source).suffix}'
    export = cv.VideoWriter(temp, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    
    # Intializing tracker
    tracker = DeepSort(embedder_gpu=False)
    
    # Initializing some helper variables.
    preds = []
    total_obj = 0

    while(True):
        ret, frame = video.read()
        if ret == True:
            # Run the ANPR algorithm
            bboxes, scores = detect_plate(frame)
            # Convert Pascal VOC detections to COCO
            bboxes = list(map(lambda bbox: pascal_voc_to_coco(bbox), bboxes))
            
            if len(bboxes) > 0:
                # Storing all the required info in a list.
                detections = [(bbox, score, 'number_plate') for bbox, score in zip(bboxes, scores)]

                # Applying tracker.
                # The tracker code flow: kalman filter -> target association(using hungarian algorithm) and appearance descriptor.
                tracks = tracker.update_tracks(detections, frame=frame)

                # Checking if tracks exist.
                for track in tracks:
                    if not track.is_confirmed() or track.time_since_update > 1:
                        continue

                    # Changing track bbox to top left, bottom right coordinates
                    bbox = [int(position) for position in list(track.to_tlbr())]
                    
                    for i in range(len(bbox)):
                        if bbox[i] < 0:
                            bbox[i] = 0

                    # Cropping the license plate and applying the OCR.
                    plate_region = crop(frame, bbox)
                    plate_text, ocr_confidence = ocr_plate(plate_region)

                    # Storing the ocr output for corresponding track id.
                    output_frame = {'track_id': track.track_id, 'ocr_txt': plate_text, 'ocr_conf': ocr_confidence}

                    # Appending track_id to list only if it does not exist in the list
                    # else looking for the current track in the list and updating the highest confidence of it.
                    if track.track_id not in list(set(pred['track_id'] for pred in preds)):
                        total_obj += 1
                        preds.append(output_frame)
                    else:
                        preds, ocr_confidence, plate_text = get_best_ocr(preds, ocr_confidence, plate_text, track.track_id)
                    
                    # Plotting the prediction.
                    frame = plot_one_box_PIL(bbox, frame, label=f'{str(track.track_id)}. {plate_text}', color=[255, 150, 0], line_thickness=3)
                    cv.imshow("frame ", frame)
                    keyexit = cv.waitKey(0)
                    if keyexit == 27:
                        break
            # Write the frame into the output file
            export.write(frame)
        else:
            break 

    # When everything done, release the video capture and video write objects
    cv.destroyAllWindows()
    video.release()
    export.release()

    # Compressing the output video for smaller size and web compatibility.
    output = f'{Path(source).stem}_detected{Path(source).suffix}'
    os.system(f'ffmpeg -y -i {temp} -c:v libx264 -b:v 5000k -minrate 1000k -maxrate 8000k -pass 1 -c:a aac -f mp4 /dev/null && ffmpeg -i {temp} -c:v libx264 -b:v 5000k -minrate 1000k -maxrate 8000k -pass 2 -c:a aac -movflags faststart {output}')
    os.system(f'rm -rf {temp} ffmpeg2pass-0.log ffmpeg2pass-0.log.mbtree')

    return output
"""


"""
def get_plates_from_webcam():
    
    # Create a VideoCapture object
    video = cv.VideoCapture(0)

    # Default resolutions of the frame are obtained. The default resolutions are system dependent.
    # We convert the resolutions from float to integer.
    width = int(video.get(cv.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv.CAP_PROP_FPS)

    # Define the codec and create VideoWriter object.
    temp = f'cam_temp.mp4'
    export = cv.VideoWriter(temp, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    
    # Intializing tracker
    tracker = DeepSort(embedder_gpu=False)
    
    # Initializing some helper variables.
    preds = []
    total_obj = 0
    fr_count = 0
    while(True):
        ret, frame = video.read()
        if ret == True:
            
            fr_count+=1
            if fr_count % 10 !=0:
                continue

            # Run the ANPR algorithm
            bboxes, scores = detect_plate(frame)
            # Convert Pascal VOC detections to COCO
            bboxes = list(map(lambda bbox: pascal_voc_to_coco(bbox), bboxes))
            
            if len(bboxes) > 0:
                # Storing all the required info in a list.
                detections = [(bbox, score, 'number_plate') for bbox, score in zip(bboxes, scores)]

                # Applying tracker.
                # The tracker code flow: kalman filter -> target association(using hungarian algorithm) and appearance descriptor.
                tracks = tracker.update_tracks(detections, frame=frame)

                # Checking if tracks exist.
                for track in tracks:
                    if not track.is_confirmed() or track.time_since_update > 1:
                        continue

                    # Changing track bbox to top left, bottom right coordinates
                    bbox = [int(position) for position in list(track.to_tlbr())]
                    
                    for i in range(len(bbox)):
                        if bbox[i] < 0:
                            bbox[i] = 0

                    # Cropping the license plate and applying the OCR.
                    plate_region = crop(frame, bbox)
                    plate_text, ocr_confidence = ocr_plate(plate_region)

                    # Storing the ocr output for corresponding track id.
                    output_frame = {'track_id': track.track_id, 'ocr_txt': plate_text, 'ocr_conf': ocr_confidence}

                    # Appending track_id to list only if it does not exist in the list
                    # else looking for the current track in the list and updating the highest confidence of it.
                    if track.track_id not in list(set(pred['track_id'] for pred in preds)):
                        total_obj += 1
                        preds.append(output_frame)
                    else:
                        preds, ocr_confidence, plate_text = get_best_ocr(preds, ocr_confidence, plate_text, track.track_id)
                    
                    # Plotting the prediction.
                    frame = plot_one_box_PIL(bbox, frame, label=f'{str(track.track_id)}. {plate_text}', color=[255, 150, 0], line_thickness=3)
                    cv.imshow("frame ", frame)
                    keyexit = cv.waitKey(0) 
                    if keyexit == 27:
                        break
            # Write the frame into the output file
            export.write(frame)
        else:
            break 

    # When everything done, release the video capture and video write objects
    cv.destroyAllWindows()
    video.release()
    export.release()

    # Compressing the output video for smaller size and web compatibility.
    output = f'cam_detected.mp4'
    os.system(f'ffmpeg -y -i {temp} -c:v libx264 -b:v 5000k -minrate 1000k -maxrate 8000k -pass 1 -c:a aac -f mp4 /dev/null && ffmpeg -i {temp} -c:v libx264 -b:v 5000k -minrate 1000k -maxrate 8000k -pass 2 -c:a aac -movflags faststart {output}')
    os.system(f'rm -rf {temp} ffmpeg2pass-0.log ffmpeg2pass-0.log.mbtree')

    return output
""";

# Running the model on a photo

In [4]:
images_n_vids_path = "C:/Users/Shahab/Downloads/Jupyter/Car-license-Plate/My-Code/Project/Dataset/Car/car-picture"
image_path = os.path.join(images_n_vids_path, "27.jpg")
#video_path = os.path.join(images_n_vids_path, "test_video_short.mp4")

savepath = "C:/Users/Shahab/Downloads/Jupyter/Car-license-Plate/My-Code/Project/Dataset/Final-model-outputs"

In [54]:
plate_image = cv.imread(image_path)
detected_plate_image = get_plates_from_image(plate_image)
cv.imwrite(os.path.join(savepath, "detected_plate.png"), detected_plate_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
['1', '7', 'TA', '2', '2', '2', '2', '8']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
['5', '8', 'Q', '2', '1', '8', '5', '7']


True

# Running the model on multiple images

In [4]:
import glob
#savepath = "C:/Users/Shahab/Downloads/Jupyter/Car-license-Plate/My-Code/Project/Dataset/Final-model-outputs2"
#imagefiles = glob.glob("C:/Users/Shahab/Downloads/Jupyter/Car-license-Plate/My-Code/Project/Dataset/Car/car-picture2/*.jpg")

savepath = "C:/Users/Shahab/Downloads/Jupyter/Car-license-Plate/My-Code/Project/Dataset/Final-model-outputs"
imagefiles = glob.glob("C:/Users/Shahab/Downloads/Jupyter/Car-license-Plate/My-Code/Project/Dataset/Car/car-for-test-model/*.jpg")
imagefiles.sort()

data = []
for filename in imagefiles:
    img = cv.imread(filename)
    data.append(img)

num_images = len(data)
num_images

9

In [5]:
counter = 1
for image in data:
    #plate_image = cv.imread(image_path)
    try:
        detected_plate_image = get_plates_from_image(image)
        cv.imwrite(os.path.join(savepath, "detected_plate%s.png" %counter), detected_plate_image)
        counter += 1
    except:
        print('Error')
        continue

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
['3', '4', 'SAD', '2', '9', '1', '2', '8']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
['5', '1', 'TA', '7', '6', '8', '2', '8']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
[

In [9]:
#detected_plate_video = get_plates_from_video(video_path)

#detected_plate_webcam = get_plates_from_webcam()